Código disponível [na minha página do Github](https://github.com/arthurkenzo/atividades_ia525)

In [2]:
import cvxpy as cp
import numpy as np
import pandas as pd
import mosek
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import time
import networkx as nx
import random


from typing import Tuple
from itertools import product

• As aulas do curso são realizadas de segunda-feira a quinta-feira, das 19h às 23h.

• Duas turmas são conduzidas em paralelo ao longo do curso em cada ano.

• Em cada turma, os horários são fixos: as aulas de cada disciplina ocorrem sempre no mesmo dia da semana
para uma mesma turma. Note que turmas diferentes podem ter a mesma disciplina em dias diferentes.

• Em cada turma, cada dia de aula está associado a uma das disciplinas do curso: matemática, português, quı́mica
e fı́sica. Cada turma deve ter uma aula de cada disciplina.

• Cada dia de aula está dividido em dois blocos: aula teórica (19h-21h) e aula de exercı́cios (21h-23h).

• Os voluntários são selecionados por meio de processo especı́fico e são distribuı́dos em três papeis:
– Professor de teoria: responsável pela aula teórica de cada disciplina.
– Professor de exercı́cios: responsável pela aula de exercı́cios de cada disciplina.
– Monitor: responsável por apoiar o professor de exercı́cios na sua aula.

• A equipe de cada disciplina conta com dois professores de exercı́cios, dois professores de teoria e monitores
que devem ser alocados nos horários e distribuı́dos entre as turmas. Para uma mesma disciplina, o número de
monitores distribuı́dos entre as turmas deve ser aproximadamente igual. Cada turma fica com um professor de
cada tipo.

• A equipe de português é menor do que as demais. Assim, nas aulas de português, as duas turmas são unidas
em uma sala. Neste caso, ambos os professores de teoria podem trabalhar como professor de exercı́cios ou
monitor no segundo horário de aula. Nas demais disciplinas, cada voluntário atua em, no máximo, uma turma.

\begin{align*}
    \text{Variáveis de decisão:} \quad &  x_{ijk}\in\mathbb{B} \hspace{20pt} \text{(Alocar voluntário } i \text{ no dia } j \text{ e horário }k\text{)} \\
                                 \quad &  y_{lm}\in\mathbb{B}  \hspace{20pt} \text{(Alocar disciplina } l \text{ no dia } m \text{)} \\

    \text{Encontrar }   & \sum_{ij} c_{ij}x_{ij}\\

    \text{sujeito a }   & \sum_{jk} x_{ijk} = 1 \quad \forall  i  \hspace{20pt} \text{(Alocar apenas um horário para cada voluntário)} \\
                        & \sum_{i} x_{ijk} = 1 \quad \forall  j,k  \hspace{20pt} \text{(Alocar apenas um voluntário em cada horário)} \\

                        & \sum_{m} y_{lm} \quad \forall l  \hspace{20pt} \text{(Apenas uma disciplina para cada dia)} \\
\end{align*}

In [88]:
# loading dataset into a pandas dataframe
availab = pd.read_csv('dados_exato.csv')
# fill empty cells with zeros
availab.fillna(0, inplace=True)
# index entries with strings
availab.index = [f'v{i}' for i in range(len(availab))]

pd.set_option('display.expand_frame_repr', False) 
print("Pre-processed DataFrame:\n", availab.head())


Pre-processed DataFrame:
    area     cargo  seg19  seg21  ter19  ter21  qua19  qua21  qui19  qui21
v0  mat  p_teoria    0.0    1.0    0.0    1.0    1.0    0.0    1.0    0.0
v1  mat   monitor    0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0
v2  mat   monitor    0.0    1.0    0.0    1.0    0.0    0.0    1.0    0.0
v3  mat   monitor    0.0    0.0    1.0    1.0    0.0    0.0    1.0    1.0
v4  mat   monitor    0.0    0.0    0.0    0.0    1.0    1.0    0.0    0.0


In [89]:
subjects = ['port', 'mat', 'fis', 'quim']
days = ['seg', 'ter', 'qua', 'qui']
shifts = ['19', '21']
groups = ['1', '2']
voluntiers = [f'v{i}' for i in range(len(availab))]

# create a dictionary with subjects as keys and voluntiers as values
voluntiersBySubject = {}
for subject in subjects:
    voluntiersBySubject[subject] = availab[availab['area'] == subject].index.tolist()

theoryProfs = availab[availab['cargo'] == 'p_teoria'].index.tolist()
practiceProfs = availab[availab['cargo'] == 'p_exerc'].index.tolist()
assistants = availab[availab['cargo'] == 'monitor'].index.tolist()
practiceVoluntiers = practiceProfs + assistants

In [94]:
nbVars = 0
# generate dictionary for indexing decision variables
allocationVars = {}
for voluntier in voluntiers:
    allocationVars[voluntier] = {}

    for day in days:
        allocationVars[voluntier][day] = {}

        for shift in shifts:
            allocationVars[voluntier][day][shift] = {}

            for group in groups:
                # do not create variables for theory profs on the second shift
                if shift == '21' and voluntier in theoryProfs:
                    break
                # do not create variables for practice profs and assistants on the first shift
                if shift == '19' and (voluntier in practiceVoluntiers):
                    break
                # generate variables only for feasible hours based on availability table
                if availab.loc[voluntier, day+shift] == 1:
                    allocationVars[voluntier][day][shift][group] = cp.Variable(boolean=True, name=f'aloc_{voluntier}_{day}_{shift}_{group}')
                    nbVars += 1

subjectIndicators = {}
for subject in subjects:
    subjectIndicators[subject] = {}
    for day in days:
        subjectIndicators[subject][day] = {}
        for group in groups:
            subjectIndicators[subject][day][group] = cp.Variable(boolean=True, name=f'indic_{subject}_{day}_{group}')


print(f"Variáveis geradas: {nbVars}")


Variáveis geradas: 216


In [100]:
def CollectNestedValues(d, target_key):
    results = []
    if isinstance(d, dict):
        for k, v in d.items():
            if k == target_key:
                results.append(v)
            if isinstance(v, dict):
                results.extend(CollectNestedValues(v, target_key))
            elif isinstance(v, list):
                for item in v:
                    results.extend(CollectNestedValues(item, target_key))
    return results

def GetAllValuesByKey(nestedDict, targetKey):
    results = []
    if isinstance(nestedDict, dict):
        for key, value in nestedDict.items():
            if key == targetKey:
                results.append(value)
            if isinstance(value, dict):
                results.extend(GetAllValuesByKey(value, targetKey))
            elif isinstance(value, list):
                for item in value:
                    results.extend(GetAllValuesByKey(item, targetKey))
    return results

def VolunteersWithVarsOn(allocationVars, targetDay, targetShift, targetGroup):
    matchingVolunteers = []

    for volunteer, dayDict in allocationVars.items():
        shiftDict = dayDict.get(targetDay, {}).get(targetShift, {})
        if targetGroup in shiftDict:
            matchingVolunteers.append(volunteer)

    return matchingVolunteers

In [ ]:
constraints = []

#TODO: faz sentido deixar a restrição como "<=" para perimitir voluntarios não alocados?
# only one shift per voluntier
for voluntier in voluntiers:
    # list of all days and shifts where the current voluntier is available
    shiftsAvailable = [allocationVars[voluntier][day][shift] 
                        for day in allocationVars.get(voluntier, {}) 
                        for shift in allocationVars[voluntier].get(day, {})]
    if shiftsAvailable:
        #TODO: fix to include groups
        # constraints += [cp.sum(shiftsAvailable) == 1]
        pass


# exactly one voluntier per shift
for day in days:
    for shift in shifts:
        for group in groups:
            # list of all voluntiers having a variable associated to the current day, shift and group
            voluntiersAvailable = VolunteersWithVarsOn(allocationVars, day, shift, group)
            if voluntiersAvailable:
                #TODO: fix to include groups
                # constraints += [cp.sum(voluntiersAvailable) == 1]
                pass


# exactly one subject per day per group
for subject in subjects:
    for group in groups:
        # constraints += [cp.sum([subjectIndicators[subject][day][group] for day in days]) == 1]
        pass


# at each group, if subject i is allocated to day j, do not allocate any voluntiers of other subjects to this day
for subject in subjects:
    for day in days:
        for group in groups:
            # collect the keys for all voluntiers that cannot teach the current subject
            prohibitedVoluntiers = [index for _subject in voluntiersBySubject 
                                    if _subject != subject 
                                    for index in voluntiersBySubject[_subject] ]
            
            # collect all alocation variables corresponding to the prohibited voluntiers 
            prohibitedAllocations = []
            for prohibitedVol in prohibitedVoluntiers:
                prohibitedAllocations.extend(GetAllValuesByKey(allocationVars[prohibitedVol], group))

            constraints += [subjectIndicators[subject][day][group] + cp.sum(prohibitedAllocations) == 1]
            pass

    

# alocate teaching assistants evenly among the practice shifts
objective = cp.Minimize(0) 


In [ ]:


# solving
lp = cp.Problem(objective, constraints)

print(lp)


minimize 0.0
subject to indic_port_seg_1 + aloc_v0_qua_19_1 + aloc_v0_qui_19_1 + aloc_v1_seg_21_1 + aloc_v2_seg_21_1 + aloc_v2_ter_21_1 + aloc_v3_ter_21_1 + aloc_v3_qui_21_1 + aloc_v4_qua_21_1 + aloc_v5_qui_21_1 + aloc_v6_seg_19_1 + aloc_v6_ter_19_1 + aloc_v6_qua_19_1 + aloc_v6_qui_19_1 + aloc_v7_seg_21_1 + aloc_v7_ter_21_1 + aloc_v7_qui_21_1 + aloc_v8_seg_21_1 + aloc_v9_seg_21_1 + aloc_v9_ter_21_1 + aloc_v9_qui_21_1 + aloc_v10_ter_21_1 + aloc_v10_qua_21_1 + aloc_v11_seg_21_1 + aloc_v11_ter_21_1 + aloc_v11_qua_21_1 + aloc_v11_qui_21_1 + aloc_v12_seg_21_1 + aloc_v12_ter_21_1 + aloc_v12_qua_21_1 + aloc_v12_qui_21_1 + aloc_v13_seg_21_1 + aloc_v13_qua_21_1 + aloc_v14_qua_21_1 + aloc_v15_seg_21_1 + aloc_v15_ter_21_1 + aloc_v15_qua_21_1 + aloc_v16_seg_21_1 + aloc_v16_ter_21_1 + aloc_v16_qua_21_1 + aloc_v30_seg_21_1 + aloc_v30_ter_21_1 + aloc_v30_qua_21_1 + aloc_v31_seg_21_1 + aloc_v31_ter_21_1 + aloc_v32_seg_19_1 + aloc_v32_ter_19_1 + aloc_v32_qua_19_1 + aloc_v32_qui_19_1 + aloc_v33_qua_21_1

In [ ]:
for subject in subjects:
    all_volunteers = set().union(*voluntiersBySubject.values())

    # Remove indexes for excluded subject
    # filtered_indexes = list(all_volunteers - set(subjectIndexes.get(subject, [])))
    filtered_indexes = [
    idx 
    for _subject in voluntiersBySubject 
    if _subject != subject 
    for idx in voluntiersBySubject[_subject]  # Keeps duplicates if they exist
]
    print(filtered_indexes)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '21', '22', '23', '24', '25', '26', '27', '28', '29']
['17', '18', '19', '20', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '21', '22', '23', '24', '25', '26', '27', '28', '29']
['17', '18', '19', '20', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '21', '22', '23', '24', '25', '26', '27', '28', '29']
['17', '18', '19', '20', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42']
